In [7]:
import src.data as data

data.DataFetcher.main()
data.PreProcess.main(False)

from run_model import ModelPred

model_preds = [
    ModelPred('gru_day'    , 'swalast' , 0 , 'gru_day_V0') ,
    ModelPred('gruRTN_day' , 'swalast' , 0 , 'gruRTN_day_V0') ,
    ModelPred('gruRES_day' , 'swalast' , 0 , 'gruRES_day_V0')
]
[md.get_df().deploy() for md in model_preds]

Load  2 DataBlocks ...... cost 0.09 secs
Align 2 DataBlocks ...... cost 0.08 secs
20231201 new
Pre-Norming method of [day] : [endpoint_division(True) , history_standardize(True)]
Load  2 DataBlocks ...... cost 0.05 secs
Align 2 DataBlocks ...... cost 0.09 secs
20231201 new
Pre-Norming method of [day] : [endpoint_division(True) , history_standardize(True)]
Load  2 DataBlocks ...... cost 0.04 secs
Align 2 DataBlocks ...... cost 0.10 secs
20231201 new
Pre-Norming method of [day] : [endpoint_division(True) , history_standardize(True)]


[None, None, None]